In [1]:
%pip install spacy_thai
%pip install gensim

^C
Note: you may need to restart the kernel to use updated packages.


: 

: 

^C



KeyboardInterrupt



In [2]:
%pip install pythainlp
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [1]:
import numpy as np 
import pandas as pd
import sys
sys.executable

'c:\\Users\\User\\OneDrive\\Desktop\\seen_jupyter\\venv\\Scripts\\python.exe'

In [59]:
# df = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\seen_jupyter\research\prachatai(TK Edition).csv')
df_train = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\seen_jupyter\research\archive\prachatai_train.csv')
df_valid = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\seen_jupyter\research\archive\prachatai_validation.csv')
df_test = pd.read_csv(r'C:\Users\User\OneDrive\Desktop\seen_jupyter\research\archive\prachatai_test.csv')
df_train.shape
df_valid.shape
df_test.shape

(6789, 17)

In [ ]:
from pythainlp.corpus.common import thai_stopwords

stop = set(thai_stopwords())
n_head = 3000
label_cols = ["politics","human_rights","quality_of_life","international","social","environment","economics","culture","labor","national_security","ict","education"]
  
def split_IO(dataframe):
    # X =  df["body_text"].fillna("").astype(str).head(n_head).tolist() #["...", "...", "..."] ลด NaN
    X =  dataframe["body_text"].fillna("").astype(str).tolist() #["...", "...", "..."] ลด NaN

    Y = dataframe[label_cols].fillna(0).astype(np.float32).to_numpy()
    print("X shape:", len(X))
    print("Y shape:", Y.shape)
    assert len(X) == Y.shape[0]
    return X, Y

X_train, Y_train = split_IO(df_train)
X_valid, Y_valid = split_IO(df_valid)

X shape: 54379
Y shape: (54379, 12)
X shape: 6721
Y shape: (6721, 12)


In [17]:
import re
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

stop = set(thai_stopwords())

# --- กรอง token ให้อยู่ในรูปที่ model ใช้ง่าย ---
_token_ok = re.compile(r"^[ก-๙a-zA-Z0-9]+$")

def clean_text(s: str) -> str:
    s = "" if s is None else str(s)
    s = s.replace("\xa0", " ").replace("\t", " ").replace("\r", " ").replace("\n", " ")
    s = normalize(s)
    s = s.replace('"', " ").replace('”', " ").replace("''", " ")   
    # ยุบช่องว่างหลายๆอันให้เหลือช่องว่างเดียว
    s = re.sub(r"\s+", " ", s).strip()
    return s
     

def preprocessing(list_text, engine="newmm", min_len=2, keep_num=True):
    tovec_list = []
    tfidf_list = []
    for text in list_text:
        text = clean_text(text)
        tokenized_data = word_tokenize(text, engine=engine)
        cleaned_text = []
        for tokenized_word in tokenized_data:
            tokenized_word = tokenized_word.strip()

            if not tokenized_word:
                continue
            if len(tokenized_word) < min_len:
                continue
            if tokenized_word in stop:
                continue
            if not _token_ok.match(tokenized_word):
                continue
            if not keep_num and tokenized_word.isdigit():
                continue
            cleaned_text.append(tokenized_word)

        tovec_list.append(cleaned_text)
        tfidf_list.append(" ".join(cleaned_text))
    return tovec_list, tfidf_list



In [18]:
# TF-IDF
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(
#     ngram_range=(1,2),   # unigram, bigram 
#     min_df=2 # at least appear in 2 documents
# )

# tovec_list, tfidf_list = preprocessing(list_txt)
# # print(tfidf_list)
# outcome = vectorizer.fit_transform(tfidf_list)
# print(outcome.shape)
# sample = outcome[:100]
# outcome_df = pd.DataFrame(sample.toarray(), columns=vectorizer.get_feature_names_out())
# outcome_df


In [30]:
# Thai2Vec
from pythainlp.word_vector import WordVector

class Thai2VecEncoder:
    def __init__(self):
        # โหลด pretrained model แค่ครั้งเดียว
        self.wv = WordVector(model_name="thai2fit_wv").get_model()
        self.dim = self.wv.vector_size

    def doc_vector(self, tokens: list):
        """
        tokens: list[str] ของข่าว 1 ชิ้น
        return: vector (dim,)
        """
        vecs = []

        for tok in tokens:
            if tok in self.wv:
                vecs.append(self.wv[tok])

        if not vecs:
            return np.zeros(self.dim, dtype=np.float32)

        return np.mean(vecs, axis=0).astype(np.float32) # เฉลี่ยตามเเกนในเเนวตั้ง 

    def transform(self, tovec_list: list):
        """
        tovec_list: list[list[str]]
        return: X shape (n_docs, dim)
        """
        return np.vstack([self.doc_vector(tokens) for tokens in tovec_list])
    
    def oov_rate(self, tovec_list, wv_model):
        total = 0
        oov = 0
        for toks in tovec_list:
            for t in toks:
                total += 1
                if t not in wv_model:
                    oov += 1
        return (oov / total) if total else 0.0


# tovec_list, _ = preprocessing(X)

# encoder = Thai2VecEncoder()

# X = encoder.transform(tovec_list)

# print(X)
# print(np.isnan(X).any(), np.isinf(X).any())
# print("row norms:", np.linalg.norm(X, axis=1))
# print("OOV rate:", encoder.oov_rate(tovec_list, encoder.wv))
# (จำนวนข่าว, 300

In [21]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score


# -------------------------
# 1) Dataset wrapper
# -------------------------
class NumpyDataset(Dataset):
    def __init__(self, X: np.ndarray, Y: np.ndarray):
        """
        X: (n_docs, input_dim)  float
        Y: (n_docs, n_labels)   0/1 float
        """
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


# -------------------------
# 2) MLP Model
# -------------------------
class MLPClassifier(nn.Module):
    def __init__(self, input_dim: int, n_labels: int, hidden_dim: int = 256, dropout: float = 0.2):
        super().__init__()
        self.net = nn.Sequential(
            # nn.Linear(input_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim // 2, n_labels)
        )

    def forward(self, x):
        return self.net(x)


# -------------------------
# 3) Train / Eval loops
# -------------------------
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * Xb.size(0)

    return total_loss / len(loader.dataset)


@torch.no_grad() # off backpropagation
def eval_one_epoch(model, loader, criterion, device, threshold=0.5):
    model.eval() # ปิด Dropout (ไม่สุ่ม) and make stable outcome
    total_loss = 0.0
    all_preds = []
    all_true = []

    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)

        logits = model(Xb)
        loss = criterion(logits, yb)
        total_loss += loss.item() * Xb.size(0)

        preds = torch.sigmoid(logits)
        pred = (preds >= threshold).int()  # 0/1 per label
        all_preds.append(pred.cpu().numpy())
        all_true.append(yb.cpu().numpy())

    all_pred = np.concatenate(all_preds, axis=0)
    all_true = np.concatenate(all_true, axis=0)
    f1_micro = f1_score(all_true, all_pred, average='micro', zero_division=0)
    f1_macro = f1_score(all_true, all_pred, average='macro', zero_division=0)

    return total_loss / len(loader.dataset), f1_micro, f1_macro, all_true, all_pred


# -------------------------
# 4) Main training function (connect to your code)
# -------------------------
def train_mlp_from_thai2vec(X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray, y_val: np.ndarray, label_names=None, batch_size=64, epochs=20, lr=1e-3, seed=42, threshold=0.5):
    np.random.seed(seed)
    torch.manual_seed(seed)

    # typing check
    assert X_train.ndim == 2 # must be 2D (n_docs, input_dim)
    assert y_train.ndim == 2  # must be 2D (n_docs, n_labels)
    assert X_train.shape[0] == y_train.shape[0] # row equal?
    assert X_train.shape[1] == X_val.shape[1]
    assert y_train.shape[1] == y_val.shape[1]
    
    n_labels = y_train.shape[1]
    if label_names is None:
        label_names = [f"label_{i}" for i in range(n_labels)]
    else:
        assert len(label_names) == n_labels # ensure label names match number of labels

    train_ds = NumpyDataset(X_train, y_train)
    val_ds = NumpyDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_dim = X_train.shape[1]    # should be 300
    model = MLPClassifier(input_dim=input_dim, n_labels=n_labels).to(device)
    pos = y_train.sum(axis=0)                 # จำนวน 1 ต่อ label
    neg = y_train.shape[0] - pos              # จำนวน 0 ต่อ label
    pos_weight = torch.tensor(neg / (pos + 1e-8), dtype=torch.float32).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # การพลาด label ที่หายาก ต้องโดนลงโทษหนักกว่าปกติ
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    best_val_acc = -1.0
    best_state = None

    for epoch in range(1, epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, f1_micro, f1_macro, y_true, y_pred = eval_one_epoch(model, val_loader, criterion, device, threshold) # evaluate with validationset 1 time 

        if f1_micro > best_val_acc:
            best_val_acc = f1_micro
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()} # best_state[k] = v.cpu().clone()

        print(f"Training epochs: {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | f1_micro={f1_micro:.4f} | f1_macro={f1_macro:.4f}")

    # load best
    assert best_state is not None
    model.load_state_dict(best_state)

    # final report
    _, _, _, y_true, y_pred = eval_one_epoch(model, val_loader, criterion, device)
    print("\nThai2Vec feed by MLP classification report (per label):")
    print(classification_report(y_true, y_pred, target_names=label_names, zero_division=0))

    return model, label_names, device


@torch.no_grad()
def predict_multilabel(texts, preprocessing, encoder, model, label_names, device, threshold=0.5):
    """
    texts: list[str]
    return:
      - pred_matrix: (n, n_labels) 0/1
      - pred_labels: list[list[str]] รายชื่อ label ที่ติด
      - proba: (n, n_labels) probability ต่อ label
    """
    tovec_list, _ = preprocessing(texts)
    X_vec = encoder.transform(tovec_list)  # (n, 300)

    X_t = torch.from_numpy(X_vec).float().to(device)
    model.eval()
    logits = model(X_t)
    proba = torch.sigmoid(logits).cpu().numpy()
    pred_matrix = (proba >= threshold).astype(int)

    pred_labels = []
    for row in pred_matrix:
        pred_labels.append([label_names[i] for i, v in enumerate(row) if v == 1])

    return pred_matrix, pred_labels, proba

# -------------------------
# 5) Usage 
# -------------------------

tovec_list, _ = preprocessing(X_train)
encoder = Thai2VecEncoder()
X_train_vec = encoder.transform(tovec_list)   # (n_docs, 300)

tovec_list2, _2 = preprocessing(X_valid)

X_val_vec = encoder.transform(tovec_list2)   # (n_docs, 300)

label_cols = [ "politics","human_rights","quality_of_life","international","social","environment","economics","culture","labor","national_security","ict","education"]
model, label_names, device = train_mlp_from_thai2vec(X_train_vec, Y_train, X_val_vec, Y_valid, label_names=label_cols)

In [63]:
test_texts = [
    "ศาลสั่งเพิกถอนคำสั่งและคุ้มครองสิทธิเสรีภาพผู้ชุมนุม",
    "เศรษฐกิจชะลอจากดอกเบี้ยสูงและราคาพลังงาน",
    "กระทรวงศึกษาฯปรับหลักสูตรใหม่และเพิ่มงบโรงเรียน"
]

pred_matrix, pred_labels, proba = predict_multilabel(
    test_texts, preprocessing, encoder, model, label_names, device, threshold=0.5
)

for t, labs in zip(test_texts, pred_labels):
    print("\nTEXT:", t)
    print("PRED:", labs)



TEXT: ศาลสั่งเพิกถอนคำสั่งและคุ้มครองสิทธิเสรีภาพผู้ชุมนุม
PRED: ['international', 'environment', 'national_security', 'ict']

TEXT: เศรษฐกิจชะลอจากดอกเบี้ยสูงและราคาพลังงาน
PRED: ['international', 'environment', 'ict']

TEXT: กระทรวงศึกษาฯปรับหลักสูตรใหม่และเพิ่มงบโรงเรียน
PRED: ['human_rights', 'international', 'environment', 'ict']
